In [1]:
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_core.prompts.prompt import PromptTemplate
from langchain.memory.buffer_window import ConversationBufferWindowMemory
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [2]:
os.environ['OPENAI_API_KEY']=os.environ.get('OPENAI_API_KEY')

In [3]:
# loading csv file
import csv
loader = DirectoryLoader('dataset/', glob="**/*.csv")
documents = loader.load()

In [4]:
# splitting the document
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, 
                                      chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [7]:
print(type(texts))

<class 'list'>


In [13]:
for text in texts:
  # Check the length of each chunk (should be around 1000 characters)
  print(len(text.page_content))

20
330
961
998
940
330
999
999
476
999
994
678
630
995
997
330
844
996
679
668
994
608
668
994
618
526
996
993
346
330
997
324
997
893
953
999
826
330
951
990
601
558
994
398
330
923
989
998
245
972
997
476
330
952
997
996
801
330
995
998
544
999
595
853
997
994
305
330
994
674
996
699
330
907
993
518
330
998
330
987
750
629
997
576
828
997
406
330
994
996
995
339
994
483
661
992
856
376
998
705
992
388
376
757
992
501
376
993
379
997
996
597
607
981
357
893
995
357
376
996
557
788
659
726
769
733
376
744
746
531
943
523
934
671
999
584
861
802
974
995
999
350
663
817
376
999
209
684
755
655
629
799
376
647
769
376
997
336
996
401
376
643
992
998
743
859
991
517
694
947
824
759
376
868
997
995
404
376
999
307
960
824
996
584
919
919
537
997
369
376
998
371
825
820
634
734
860
376
982
934
376
741
661
740
998
355
718
708
820
986
805
376
909
794
728
819
979
893
804
999
470
376
999
611
997
689
760
608
376
996
830
992
995
252
376
907
705
376
821
572
376
659
997
310
182
986
832
182
997
713
7

In [ ]:
# embeddings = OpenAIEmbeddings()

In [14]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
# load it into Chroma
db = Chroma.from_documents(texts, embedding_function)

/home/employee/Desktop/Hachathon/mha-llm/hackvenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/employee/Desktop/Hachathon/mha-llm/hackvenv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
# query it
query = "How do I handle work pressure?"
docs = db.similarity_search(query)

In [19]:
# print results
print(docs[0].page_content)

colleagues or supervisor if you're feeling overwhelmed. Communicating openly about your concerns may help alleviate some of the pressure.\n5. Set boundaries: Be clear about your work schedule and set boundaries about when and how often you're available to work. This can help you establish a better work-life balance and reduce the risk of burnout.\n6. Seek support: Reach out to friends, family, or a mental health professional if you're feeling particularly demoralized or overwhelmed. They can offer a fresh perspective and help you develop coping strategies to manage your stress.


In [17]:
# save to disk
db2 = Chroma.from_documents(docs, embedding_function, persist_directory="./chroma_db")
docs = db2.similarity_search(query)

In [20]:
# load from disk
db3 = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)
docs = db3.similarity_search(query)
print(docs[0].page_content)

in your life. Consider reaching out to trusted friends, family members, or community resources (such as local mental health clinics) for additional support and resources.\n2. Engage in calming activities: Anxiety can often be exacerbated by a lack of structure and routine. Try to engage in activities that help calm your mind and reduce stress. This might include things like taking a warm bath, practicing mindfulness meditation, engaging in physical activity (such as going for a walk), or reading a book.\n3. Implement coping skills: While you're waiting to connect with your therapist or other mental health professionals, it can be helpful to practice coping skills that can help you manage your anxiety when it arises. Some common coping skills include deep breathing exercises, progressive muscle relaxation, visualization techniques, and cognitive restructuring.\n4. Consider seeking emergency services: If you're experiencing severe anxiety, suicidal thoughts, or self-harm, it's important


In [21]:
import chromadb

persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection("collection_name")
collection.add(ids=["1", "2", "3"], documents=["a", "b", "c"])

langchain_chroma = Chroma(
    client=persistent_client,
    collection_name="collection_name",
    embedding_function=embedding_function,
)

print("There are", langchain_chroma._collection.count(), "in the collection")

/home/employee/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [03:42<00:00, 374kiB/s] 


There are 3 in the collection
